In [1]:
# Imports
from pandas_datareader import data as pdr
from yahoo_fin import stock_info as si
from pandas import ExcelWriter
import yfinance as yf
import pandas as pd
import datetime
import time
yf.pdr_override()
from pathlib import Path
import numpy as np

In [2]:
# scan a list of stocks from a csv file of Investors Business Daily recommendations
csv_path = Path("IBDStocks.csv")
ibd_df = pd.read_csv(csv_path)

In [3]:
ibd_df.tail()

,Symbol
63,TSLA
64,FB
65,AAPL
66,LOW
67,AMZN


In [4]:
# Variables, tickers from the IBD list
#tickers = si.tickers_sp500()
tickers = ibd_df["Symbol"]
tickers = [item.replace(".", "-") for item in tickers] # Yahoo Finance uses dashes instead of dots
index_name = 'IBD' # S&P 500
exportList = pd.DataFrame(columns=['Stock', "RS_Rating", "50 Day MA", "150 Day Ma", "200 Day MA", "52 Week Low", "52 week High"])
returns_multiples = []

In [5]:
#Index from the list above Returns
index_df = pdr.get_data_yahoo(index_name)
index_df['Percent Change'] = index_df['Adj Close'].pct_change()
index_return = (index_df['Percent Change'] + 1).cumprod()[-1]

[*********************100%***********************]  1 of 1 completed


In [6]:
# parameter setup (default values in the original indicator)
#These are the constants to be used for the moving averages, the bollinger bands and the keltner channels
#ultimately these will be needed for the squeeze and other calculations
length = 20
mult = 2
length_KC = 21
mult_KC = 2

In [7]:
for ticker in tickers:
    # Download historical data as CSV for each stock (makes the process faster)
    stock = yf.Ticker(ticker)
    df = stock.history(period="3y", interval='1d')
    df.to_csv(f'{ticker}.csv')    
    m_avg = df['Close'].ewm(span=length).mean()
    df['Moving average'] = m_avg
    # standard deviation
    m_std = df['Close'].rolling(window=length).std()
    # upper Bollinger Bands
    df['upper_BB'] = m_avg + mult * m_std
    # lower Bollinger Bands 
    df['lower_BB'] = m_avg - mult * m_std
    # calculate Keltner Channel which is a measure of the average true range.  In this case the channels are set to twice the average true range
    # first we need to calculate True Range
    df['tr0'] = abs(df["High"] - df["Low"])
    df['tr1'] = abs(df["High"] - df["Close"].shift())
    df['tr2'] = abs(df["Low"] - df["Close"].shift())
    df['tr'] = df[['tr0', 'tr1', 'tr2']].max(axis=1)
    # moving average of the TR or average true range
    range_ma = df['tr'].ewm(span=length_KC).mean()
    # upper Keltner Channel
    df['upper_KC'] = m_avg + range_ma * mult_KC
    # lower Keltner Channel
    df['lower_KC'] = m_avg - range_ma * mult_KC
    
    df['squeeze_on'] = (df['lower_BB'] > df['lower_KC']) & (df['upper_BB'] < df['upper_KC'])
    df['squeeze_off'] = (df['lower_BB'] < df['lower_KC']) & (df['upper_BB'] > df['upper_KC'])
    
    highest = df['High'].rolling(window = length_KC).max()
    lowest = df['Low'].rolling(window = length_KC).min()
    m1 = (highest + lowest) / 2
    df['value'] = (df['Close'] - (m1 + m_avg)/2)
    fit_y = np.array(range(0,length_KC))
    df['value'] = df['value'].rolling(window = length_KC).apply(lambda x : np.polyfit(fit_y, x, 1)[0] * (length_KC-1) +
    np.polyfit(fit_y, x, 1)[1], raw=True)

    df[['squeeze_on', 'squeeze_off']] = (df[['squeeze_on', 'squeeze_off']] == True).astype(float)

    df["pct_change"] = df['Close'].pct_change()*100

    conditions =  [
        (df['pct_change'] > 0),
        (df['pct_change'] < 0)
    ]

    label = [1, 0]
    df['profitable?'] = np.select(conditions, label) 
    
    # entry point for long position:
    # 1. black cross becomes gray (the squeeze is released)
    long_cond1 = (df['squeeze_off'][-2] == 0) | (df['squeeze_off'][-1] == 1) 
    # 2. bar value is positive => the bar is light green
    long_cond2 = df['value'][-1] > 0

    enter_long = long_cond1 and long_cond2
    # entry point for short position:
    # 1. black cross becomes gray (the squeeze is released)
    short_cond1 = (df['squeeze_off'][-2] == 0) | (df['squeeze_off'][-1] == 1) 
    # 2. bar value is negative => the bar is light red 
    short_cond2 = df['value'][-1] < 0
    enter_short = short_cond1 and short_cond2

    colors = []
    for ind, val in enumerate(df['value']):
        if val >= 0:
            color = 'blue'
            if val > df['value'][ind-1]:
                color = 'cyan'
                    
        else:
            color = 'yellow'
            if val < df['value'][ind-1]:
                color='red'        
        colors.append(color)    
    df['colors'] = colors
    color_value = df['colors'][-1]
    squeeze_value = df['squeeze_on'][-1]
    
    print (f'Ticker: {ticker}; Squeeze_scan\n {squeeze_value},{color_value}')
    time.sleep(1)

Ticker: INMD; Squeeze_scan
 1.0,blue
Ticker: ASAN; Squeeze_scan
 1.0,cyan
Ticker: ATKR; Squeeze_scan
 0.0,cyan
Ticker: SWAV; Squeeze_scan
 0.0,red
Ticker: LSPD; Squeeze_scan
 0.0,red
Ticker: CELH; Squeeze_scan
 0.0,red
Ticker: MRNA; Squeeze_scan
 0.0,yellow
Ticker: BILL; Squeeze_scan
 0.0,blue
Ticker: UPST; Squeeze_scan
 0.0,red
Ticker: TASK; Squeeze_scan
 1.0,red
Ticker: NET; Squeeze_scan
 0.0,blue
Ticker: SE; Squeeze_scan
 1.0,red
Ticker: DOCS; Squeeze_scan
 1.0,blue
Ticker: ASML; Squeeze_scan
 0.0,blue
Ticker: WAL; Squeeze_scan
 1.0,red
Ticker: GNRC; Squeeze_scan
 0.0,red
Ticker: MDB; Squeeze_scan
 0.0,blue
Ticker: GLBE; Squeeze_scan
 0.0,cyan
Ticker: DLO; Squeeze_scan
 0.0,red
Ticker: RGEN; Squeeze_scan
 0.0,yellow
Ticker: PRFT; Squeeze_scan
 0.0,blue
Ticker: ZS; Squeeze_scan
 0.0,cyan
Ticker: AVTR; Squeeze_scan
 1.0,cyan
Ticker: DDOG; Squeeze_scan
 0.0,blue
Ticker: SITM; Squeeze_scan
 0.0,cyan
Ticker: SPT; Squeeze_scan
 1.0,blue
Ticker: MNDY; Squeeze_scan
 1.0,red
Ticker: WST; Squ

In [10]:
for ticker in tickers:
    # Download historical data as CSV for each stock (makes the process faster)
    stock = yf.Ticker(ticker)
    df = stock.history(period="3y", interval='1d')
    df.to_csv(f'{ticker}.csv')
    #Stacked Moving averages, exponentially weighted.  Focusing on Fibonacci numbers here with 8, 21, 34, 55 and 89 day moving averages
    m_avg_08 = df['Close'].ewm(span=8).mean()
    m_avg_21 = df['Close'].ewm(span=21).mean()
    m_avg_34 = df['Close'].ewm(span=34).mean()
    m_avg_55 = df['Close'].ewm(span=55).mean()
    m_avg_89 = df['Close'].ewm(span=89).mean()

    df['m_avg_08'] = m_avg_08
    df['m_avg_21'] = m_avg_21
    df['m_avg_34'] = m_avg_34
    df['m_avg_55'] = m_avg_55
    df['m_avg_89'] = m_avg_89
    
    df['tr0'] = abs(df["High"] - df["Low"])
    df['tr1'] = abs(df["High"] - df["Close"].shift())
    df['tr2'] = abs(df["Low"] - df["Close"].shift())
    df['tr'] = df[['tr0', 'tr1', 'tr2']].max(axis=1)
    # moving average of the TR or average true range
    range_ma = df['tr'].ewm(span=length_KC).mean()
    # upper Keltner Channel
    df['upper_KC'] = m_avg + range_ma * mult_KC
    # lower Keltner Channel
    df['lower_KC'] = m_avg - range_ma * mult_KC
    
    plus_dm = df["High"].diff()
    minus_dm = df["Low"].diff()
    plus_dm[plus_dm < 0] = 0
    minus_dm[minus_dm > 0] = 0

    #plus_di and minus di show the direction of the trend, if plus_di > minus_di the trend is positive
    #if minus_di > plus_di the trend is negative
    df['atr'] = df['tr'].ewm(span=14).mean()
    df['plus_di'] = 100 * (plus_dm.ewm(alpha = 1/14).mean() / df['atr'])
    df['minus_di'] = abs(100 * (minus_dm.ewm(alpha = 1/14).mean() / df['atr']))
    dx = (abs(df['plus_di'] - df['minus_di']) / abs(df['plus_di'] + df['minus_di'])) * 100
    adx = ((dx.shift(1) * (14 - 1)) + dx) / 14
    df['adx'] = adx.ewm(alpha = 1/14).mean() #ADX>20 is trending ADX<20 not trending
    
    df['ADX_trend'] = 20
    
    
    
    if (df['m_avg_08'][-1] > df['m_avg_21'][-1] > df['m_avg_34'][-1] > df['m_avg_55'][-1] > df['m_avg_89'][-1]):
        print (f'Ticker: {ticker}; Stacked EMA\n is in the positive direction')
    elif (df['m_avg_08'][-1] < df['m_avg_21'][-1] < df['m_avg_34'][-1] < df['m_avg_55'][-1] < df['m_avg_89'][-1]):
        print (f'Ticker: {ticker}; Stacked EMA\n is in the negative direction')
    else:
        print (f'Ticker: {ticker}; Stacked EMA\n is neutral')
        
    if (df['adx'][-1] > df['ADX_trend'][-1]) and (df['plus_di'][-1] > df['minus_di'][-1]):
        print (f'Ticker: {ticker}; Directional trend is positive')
    elif (df['adx'][-1] > df['ADX_trend'][-1]) and (df['plus_di'][-1] < df['minus_di'][-1]):
        print (f'Ticker: {ticker}; Directional trend is negative')
    elif (df['adx'][-1] < df['ADX_trend'][-1]):
        print (f'Ticker: {ticker}; Directional trend is neutral or sideways')
    time.sleep(1)

Ticker: INMD; Stacked EMA
 is in the positive direction
Ticker: INMD; Directional trend is neutral or sideways
Ticker: ASAN; Stacked EMA
 is in the positive direction
Ticker: ASAN; Directional trend is positive
Ticker: ATKR; Stacked EMA
 is in the positive direction
Ticker: ATKR; Directional trend is positive
Ticker: SWAV; Stacked EMA
 is neutral
Ticker: SWAV; Directional trend is neutral or sideways
Ticker: LSPD; Stacked EMA
 is in the negative direction
Ticker: LSPD; Directional trend is negative
Ticker: CELH; Stacked EMA
 is neutral
Ticker: CELH; Directional trend is negative
Ticker: MRNA; Stacked EMA
 is in the negative direction
Ticker: MRNA; Directional trend is negative
Ticker: BILL; Stacked EMA
 is in the positive direction
Ticker: BILL; Directional trend is positive
Ticker: UPST; Stacked EMA
 is neutral
Ticker: UPST; Directional trend is negative
Ticker: TASK; Stacked EMA
 is neutral
Ticker: TASK; Directional trend is neutral or sideways
Ticker: NET; Stacked EMA
 is in the pos

In [11]:
for ticker in tickers:
    # Download historical data as CSV for each stock (makes the process faster)
    stock = yf.Ticker(ticker)
    df_wk = stock.history(period="5y", interval='1wk')
    df_wk.to_csv(f'{ticker}_wk.csv')    
    m_avg_wk = df_wk['Close'].ewm(span=length).mean()
    df_wk['Moving average'] = m_avg_wk
    # standard deviation
    m_std_wk = df_wk['Close'].rolling(window=length).std()
    # upper Bollinger Bands
    df_wk['upper_BB'] = m_avg_wk + mult * m_std_wk
    # lower Bollinger Bands 
    df_wk['lower_BB'] = m_avg_wk - mult * m_std_wk
    # calculate Keltner Channel which is a measure of the average true range.  In this case the channels are set to twice the average true range
    # first we need to calculate True Range
    df_wk['tr0'] = abs(df_wk["High"] - df_wk["Low"])
    df_wk['tr1'] = abs(df_wk["High"] - df_wk["Close"].shift())
    df_wk['tr2'] = abs(df_wk["Low"] - df_wk["Close"].shift())
    df_wk['tr'] = df_wk[['tr0', 'tr1', 'tr2']].max(axis=1)
    # moving average of the TR or average true range
    range_ma_wk = df_wk['tr'].ewm(span=length_KC).mean()
    # upper Keltner Channel
    df_wk['upper_KC'] = m_avg_wk + range_ma_wk * mult_KC
    # lower Keltner Channel
    df_wk['lower_KC'] = m_avg_wk - range_ma_wk * mult_KC
    
    df_wk['squeeze_on'] = (df_wk['lower_BB'] > df_wk['lower_KC']) & (df_wk['upper_BB'] < df_wk['upper_KC'])
    df_wk['squeeze_off'] = (df_wk['lower_BB'] < df_wk['lower_KC']) & (df_wk['upper_BB'] > df_wk['upper_KC'])
    
    highest_wk = df_wk['High'].rolling(window = length_KC).max()
    lowest_wk = df_wk['Low'].rolling(window = length_KC).min()
    m1_wk = (highest_wk + lowest_wk) / 2
    df_wk['value'] = (df_wk['Close'] - (m1_wk + m_avg_wk)/2)
    fit_y_wk = np.array(range(0,length_KC))
    df_wk['value'] = df_wk['value'].rolling(window = length_KC).apply(lambda x : np.polyfit(fit_y_wk, x, 1)[0] * (length_KC-1) +
    np.polyfit(fit_y_wk, x, 1)[1], raw=True)

    df_wk[['squeeze_on', 'squeeze_off']] = (df_wk[['squeeze_on', 'squeeze_off']] == True).astype(float)

    df_wk["pct_change"] = df_wk['Close'].pct_change()*100

    conditions_wk =  [
        (df_wk['pct_change'] > 0),
        (df_wk['pct_change'] < 0)
    ]

    label = [1, 0]
    df_wk['profitable?'] = np.select(conditions_wk, label) 
    
    # entry point for long position:
    # 1. black cross becomes gray (the squeeze is released)
    long_cond1_wk = (df_wk['squeeze_off'][-2] == 0) | (df_wk['squeeze_off'][-1] == 1) 
    # 2. bar value is positive => the bar is light green
    long_cond2_wk = df_wk['value'][-1] > 0

    enter_long_wk = long_cond1_wk and long_cond2_wk
    # entry point for short position:
    # 1. black cross becomes gray (the squeeze is released)
    short_cond1_wk = (df_wk['squeeze_off'][-2] == 0) | (df_wk['squeeze_off'][-1] == 1) 
    # 2. bar value is negative => the bar is light red 
    short_cond2_wk = df_wk['value'][-1] < 0
    enter_short_wk = short_cond1_wk and short_cond2_wk

    colors = []
    for ind, val in enumerate(df_wk['value']):
        if val >= 0:
            color = 'blue'
            if val > df_wk['value'][ind-1]:
                color = 'cyan'
                    
        else:
            color = 'yellow'
            if val < df_wk['value'][ind-1]:
                color='red'        
        colors.append(color)    
    df_wk['colors'] = colors
    color_value_wk = df_wk['colors'][-1]
    squeeze_value_wk = df_wk['squeeze_on'][-1]
    
    print (f'Ticker: {ticker}; weekly squeeze_scan\n {squeeze_value_wk},{color_value_wk}')
    time.sleep(1)

Ticker: INMD; weekly squeeze_scan
 0.0,yellow
Ticker: ASAN; weekly squeeze_scan
 0.0,blue
Ticker: ATKR; weekly squeeze_scan
 0.0,cyan
Ticker: SWAV; weekly squeeze_scan
 1.0,blue
Ticker: LSPD; weekly squeeze_scan
 0.0,red
Ticker: CELH; weekly squeeze_scan
 0.0,blue
Ticker: MRNA; weekly squeeze_scan
 0.0,red
Ticker: BILL; weekly squeeze_scan
 0.0,blue
Ticker: UPST; weekly squeeze_scan
 0.0,blue
Ticker: TASK; weekly squeeze_scan
 0.0,yellow
Ticker: NET; weekly squeeze_scan
 0.0,cyan
Ticker: SE; weekly squeeze_scan
 1.0,blue
Ticker: DOCS; weekly squeeze_scan
 0.0,yellow
Ticker: ASML; weekly squeeze_scan
 0.0,yellow
Ticker: WAL; weekly squeeze_scan
 0.0,yellow
Ticker: GNRC; weekly squeeze_scan
 1.0,blue
Ticker: MDB; weekly squeeze_scan
 0.0,cyan
Ticker: GLBE; weekly squeeze_scan
 1.0,yellow
Ticker: DLO; weekly squeeze_scan
 0.0,yellow
Ticker: RGEN; weekly squeeze_scan
 0.0,blue
Ticker: PRFT; weekly squeeze_scan
 0.0,cyan
Ticker: ZS; weekly squeeze_scan
 0.0,cyan
Ticker: AVTR; weekly squeeze

In [15]:
for ticker in tickers:
    # Download historical data as CSV for each stock (makes the process faster)
    stock = yf.Ticker(ticker)
    df_wk = stock.history(period="5y", interval='1wk')
    df_wk.to_csv(f'{ticker}_wk.csv')
    #Stacked Moving averages, exponentially weighted.  Focusing on Fibonacci numbers here with 8, 21, 34, 55 and 89 day moving averages
    m_avg_08_wk = df_wk['Close'].ewm(span=8).mean()
    m_avg_21_wk = df_wk['Close'].ewm(span=21).mean()
    m_avg_34_wk = df_wk['Close'].ewm(span=34).mean()
    m_avg_55_wk = df_wk['Close'].ewm(span=55).mean()
    m_avg_89_wk = df_wk['Close'].ewm(span=89).mean()

    df_wk['m_avg_08'] = m_avg_08_wk
    df_wk['m_avg_21'] = m_avg_21_wk
    df_wk['m_avg_34'] = m_avg_34_wk
    df_wk['m_avg_55'] = m_avg_55_wk
    df_wk['m_avg_89'] = m_avg_89_wk
    
    df_wk['tr0'] = abs(df_wk["High"] - df_wk["Low"])
    df_wk['tr1'] = abs(df_wk["High"] - df_wk["Close"].shift())
    df_wk['tr2'] = abs(df_wk["Low"] - df_wk["Close"].shift())
    df_wk['tr'] = df_wk[['tr0', 'tr1', 'tr2']].max(axis=1)
    # moving average of the TR or average true range
    range_ma = df_wk['tr'].ewm(span=length_KC).mean()
    # upper Keltner Channel
    df_wk['upper_KC'] = m_avg_wk + range_ma * mult_KC
    # lower Keltner Channel
    df_wk['lower_KC'] = m_avg_wk - range_ma * mult_KC
    
    plus_dm = df_wk["High"].diff()
    minus_dm = df_wk["Low"].diff()
    plus_dm[plus_dm < 0] = 0
    minus_dm[minus_dm > 0] = 0

    #plus_di and minus di show the direction of the trend, if plus_di > minus_di the trend is positive
    #if minus_di > plus_di the trend is negative
    df_wk['atr'] = df_wk['tr'].ewm(span=14).mean()
    df_wk['plus_di'] = 100 * (plus_dm.ewm(alpha = 1/14).mean() / df_wk['atr'])
    df_wk['minus_di'] = abs(100 * (minus_dm.ewm(alpha = 1/14).mean() / df_wk['atr']))
    dx_wk = (abs(df_wk['plus_di'] - df_wk['minus_di']) / abs(df_wk['plus_di'] + df_wk['minus_di'])) * 100
    adx_wk = ((dx_wk.shift(1) * (14 - 1)) + dx_wk) / 14
    df_wk['adx'] = adx_wk.ewm(alpha = 1/14).mean() #ADX>20 is trending ADX<20 not trending
    
    df_wk['ADX_trend'] = 20
    
    
    
    if (df_wk['m_avg_08'][-1] > df_wk['m_avg_21'][-1] > df_wk['m_avg_34'][-1] > df_wk['m_avg_55'][-1] > df_wk['m_avg_89'][-1]):
        print (f'Ticker: {ticker}; Weekly stacked EMA\n is in the positive direction')
    elif (df_wk['m_avg_08'][-1] < df_wk['m_avg_21'][-1] < df_wk['m_avg_34'][-1] < df_wk['m_avg_55'][-1] < df_wk['m_avg_89'][-1]):
        print (f'Ticker: {ticker}; Weekly stacked EMA\n is in the negative direction')
    else:
        print (f'Ticker: {ticker}; Weekly stacked EMA\n is neutral')
        
    if (df_wk['adx'][-1] > df_wk['ADX_trend'][-1]) and (df_wk['plus_di'][-1] > df_wk['minus_di'][-1]):
        print (f'Ticker: {ticker}; Weekly directional trend is positive')
    elif (df_wk['adx'][-1] > df_wk['ADX_trend'][-1]) and (df_wk['plus_di'][-1] < df_wk['minus_di'][-1]):
        print (f'Ticker: {ticker}; Directional trend is negative')
    elif (df_wk['adx'][-1] < df_wk['ADX_trend'][-1]):
        print (f'Ticker: {ticker}; Directional trend is neutral or sideways')
    time.sleep(1)

Ticker: INMD; Weekly stacked EMA
 is in the positive direction
Ticker: INMD; Weekly directional trend is positive
Ticker: ASAN; Weekly stacked EMA
 is in the positive direction
Ticker: ASAN; Weekly directional trend is positive
Ticker: ATKR; Weekly stacked EMA
 is in the positive direction
Ticker: ATKR; Weekly directional trend is positive
Ticker: SWAV; Weekly stacked EMA
 is in the positive direction
Ticker: SWAV; Weekly directional trend is positive
Ticker: LSPD; Weekly stacked EMA
 is neutral
Ticker: LSPD; Directional trend is negative
Ticker: CELH; Weekly stacked EMA
 is in the positive direction
Ticker: CELH; Weekly directional trend is positive
Ticker: MRNA; Weekly stacked EMA
 is neutral
Ticker: MRNA; Directional trend is negative
Ticker: BILL; Weekly stacked EMA
 is in the positive direction
Ticker: BILL; Weekly directional trend is positive
Ticker: UPST; Weekly stacked EMA
 is in the positive direction
Ticker: UPST; Weekly directional trend is positive
Ticker: TASK; Weekly sta

In [16]:
for ticker in tickers:
    # Download historical data as CSV for each stock (makes the process faster)
    stock = yf.Ticker(ticker)
    df_mo = stock.history(period="10y", interval='1mo')
    df_mo.to_csv(f'{ticker}_mo.csv')    
    m_avg_mo = df_mo['Close'].ewm(span=length).mean()
    df_mo['Moving average'] = m_avg_mo
    # standard deviation
    m_std_mo = df_mo['Close'].rolling(window=length).std()
    # upper Bollinger Bands
    df_mo['upper_BB'] = m_avg_mo + mult * m_std_mo
    # lower Bollinger Bands 
    df_mo['lower_BB'] = m_avg_mo - mult * m_std_mo
    # calculate Keltner Channel which is a measure of the average true range.  In this case the channels are set to twice the average true range
    # first we need to calculate True Range
    df_mo['tr0'] = abs(df_mo["High"] - df_mo["Low"])
    df_mo['tr1'] = abs(df_mo["High"] - df_mo["Close"].shift())
    df_mo['tr2'] = abs(df_mo["Low"] - df_mo["Close"].shift())
    df_mo['tr'] = df_mo[['tr0', 'tr1', 'tr2']].max(axis=1)
    # moving average of the TR or average true range
    range_ma_mo = df_mo['tr'].ewm(span=length_KC).mean()
    # upper Keltner Channel
    df_mo['upper_KC'] = m_avg_mo + range_ma_mo * mult_KC
    # lower Keltner Channel
    df_mo['lower_KC'] = m_avg_mo - range_ma_mo * mult_KC
    
    df_mo['squeeze_on'] = (df_mo['lower_BB'] > df_mo['lower_KC']) & (df_mo['upper_BB'] < df_mo['upper_KC'])
    df_mo['squeeze_off'] = (df_mo['lower_BB'] < df_mo['lower_KC']) & (df_mo['upper_BB'] > df_mo['upper_KC'])
    
    highest_mo = df_mo['High'].rolling(window = length_KC).max()
    lowest_mo = df_mo['Low'].rolling(window = length_KC).min()
    m1_mo = (highest_mo + lowest_mo) / 2
    df_mo['value'] = (df_mo['Close'] - (m1_mo + m_avg_mo)/2)
    fit_y_mo = np.array(range(0,length_KC))
    df_mo['value'] = df_mo['value'].rolling(window = length_KC).apply(lambda x : np.polyfit(fit_y_mo, x, 1)[0] * (length_KC-1) +
    np.polyfit(fit_y_mo, x, 1)[1], raw=True)

    df_mo[['squeeze_on', 'squeeze_off']] = (df_mo[['squeeze_on', 'squeeze_off']] == True).astype(float)

    df_mo["pct_change"] = df_mo['Close'].pct_change()*100

    conditions_mo =  [
        (df_mo['pct_change'] > 0),
        (df_mo['pct_change'] < 0)
    ]

    label = [1, 0]
    df_mo['profitable?'] = np.select(conditions_mo, label) 
    
    # entry point for long position:
    # 1. black cross becomes gray (the squeeze is released)
    long_cond1_mo = (df_mo['squeeze_off'][-2] == 0) | (df_mo['squeeze_off'][-1] == 1) 
    # 2. bar value is positive => the bar is light green
    long_cond2_mo = df_mo['value'][-1] > 0

    enter_long_mo = long_cond1_mo and long_cond2_mo
    # entry point for short position:
    # 1. black cross becomes gray (the squeeze is released)
    short_cond1_mo = (df_mo['squeeze_off'][-2] == 0) | (df_mo['squeeze_off'][-1] == 1) 
    # 2. bar value is negative => the bar is light red 
    short_cond2_mo = df_mo['value'][-1] < 0
    enter_short_mo = short_cond1_mo and short_cond2_mo

    colors = []
    for ind, val in enumerate(df_mo['value']):
        if val >= 0:
            color = 'blue'
            if val > df_mo['value'][ind-1]:
                color = 'cyan'
                    
        else:
            color = 'yellow'
            if val < df_mo['value'][ind-1]:
                color='red'        
        colors.append(color)    
    df_mo['colors'] = colors
    color_value_mo = df_mo['colors'][-1]
    squeeze_value_mo = df_mo['squeeze_on'][-1]
    
    print (f'Ticker: {ticker}; monthly squeeze_scan\n {squeeze_value_mo},{color_value_mo}')
    time.sleep(1)

Ticker: INMD; monthly squeeze_scan
 0.0,yellow
Ticker: ASAN; monthly squeeze_scan
 0.0,yellow
Ticker: ATKR; monthly squeeze_scan
 0.0,cyan
Ticker: SWAV; monthly squeeze_scan
 0.0,yellow
Ticker: LSPD; monthly squeeze_scan
 0.0,yellow
Ticker: CELH; monthly squeeze_scan
 0.0,blue
Ticker: MRNA; monthly squeeze_scan
 0.0,yellow
Ticker: BILL; monthly squeeze_scan
 0.0,yellow
Ticker: UPST; monthly squeeze_scan
 0.0,yellow
Ticker: TASK; monthly squeeze_scan
 0.0,yellow
Ticker: NET; monthly squeeze_scan
 0.0,yellow
Ticker: SE; monthly squeeze_scan
 0.0,blue
Ticker: DOCS; monthly squeeze_scan
 0.0,yellow
Ticker: ASML; monthly squeeze_scan
 0.0,yellow
Ticker: WAL; monthly squeeze_scan
 0.0,yellow
Ticker: GNRC; monthly squeeze_scan
 0.0,blue
Ticker: MDB; monthly squeeze_scan
 0.0,cyan
Ticker: GLBE; monthly squeeze_scan
 0.0,yellow
Ticker: DLO; monthly squeeze_scan
 0.0,yellow
Ticker: RGEN; monthly squeeze_scan
 0.0,blue
Ticker: PRFT; monthly squeeze_scan
 0.0,cyan
Ticker: ZS; monthly squeeze_scan


In [17]:
for ticker in tickers:
    # Download historical data as CSV for each stock (makes the process faster)
    stock = yf.Ticker(ticker)
    df_mo = stock.history(period="10y", interval='1mo')
    df_mo.to_csv(f'{ticker}_mo.csv')
    #Stacked Moving averages, exponentially weighted.  Focusing on Fibonacci numbers here with 8, 21, 34, 55 and 89 day moving averages
    m_avg_08_mo = df_mo['Close'].ewm(span=8).mean()
    m_avg_21_mo = df_mo['Close'].ewm(span=21).mean()
    m_avg_34_mo = df_mo['Close'].ewm(span=34).mean()
    m_avg_55_mo = df_mo['Close'].ewm(span=55).mean()
    m_avg_89_mo = df_mo['Close'].ewm(span=89).mean()

    df_mo['m_avg_08'] = m_avg_08_mo
    df_mo['m_avg_21'] = m_avg_21_mo
    df_mo['m_avg_34'] = m_avg_34_mo
    df_mo['m_avg_55'] = m_avg_55_mo
    df_mo['m_avg_89'] = m_avg_89_mo
    
    df_mo['tr0'] = abs(df_mo["High"] - df_mo["Low"])
    df_mo['tr1'] = abs(df_mo["High"] - df_mo["Close"].shift())
    df_mo['tr2'] = abs(df_mo["Low"] - df_mo["Close"].shift())
    df_mo['tr'] = df_mo[['tr0', 'tr1', 'tr2']].max(axis=1)
    # moving average of the TR or average true range
    range_ma = df_mo['tr'].ewm(span=length_KC).mean()
    # upper Keltner Channel
    df_mo['upper_KC'] = m_avg_mo + range_ma * mult_KC
    # lower Keltner Channel
    df_mo['lower_KC'] = m_avg_mo - range_ma * mult_KC
    
    plus_dm = df_mo["High"].diff()
    minus_dm = df_mo["Low"].diff()
    plus_dm[plus_dm < 0] = 0
    minus_dm[minus_dm > 0] = 0

    #plus_di and minus di show the direction of the trend, if plus_di > minus_di the trend is positive
    #if minus_di > plus_di the trend is negative
    df_mo['atr'] = df_mo['tr'].ewm(span=14).mean()
    df_mo['plus_di'] = 100 * (plus_dm.ewm(alpha = 1/14).mean() / df_mo['atr'])
    df_mo['minus_di'] = abs(100 * (minus_dm.ewm(alpha = 1/14).mean() / df_mo['atr']))
    dx_mo = (abs(df_mo['plus_di'] - df_mo['minus_di']) / abs(df_mo['plus_di'] + df_mo['minus_di'])) * 100
    adx_mo = ((dx_mo.shift(1) * (14 - 1)) + dx_mo) / 14
    df_mo['adx'] = adx_mo.ewm(alpha = 1/14).mean() #ADX>20 is trending ADX<20 not trending
    
    df_mo['ADX_trend'] = 20
    
    
    if (df_mo['m_avg_08'][-1] > df_mo['m_avg_21'][-1] > df_mo['m_avg_34'][-1] > df_mo['m_avg_55'][-1] > df_mo['m_avg_89'][-1]):
        print (f'Ticker: {ticker}; Monthly stacked EMA\n is in the positive direction')
    elif (df_mo['m_avg_08'][-1] < df_mo['m_avg_21'][-1] < df_mo['m_avg_34'][-1] < df_mo['m_avg_55'][-1] < df_mo['m_avg_89'][-1]):
        print (f'Ticker: {ticker}; Monthly stacked EMA\n is in the negative direction')
    else:
        print (f'Ticker: {ticker}; Weekly stacked EMA\n is neutral')
        
    if (df_mo['adx'][-1] > df_mo['ADX_trend'][-1]) and (df_mo['plus_di'][-1] > df_mo['minus_di'][-1]):
        print (f'Ticker: {ticker}; Monthly directional trend is positive')
    elif (df_mo['adx'][-1] > df_mo['ADX_trend'][-1]) and (df_mo['plus_di'][-1] < df_mo['minus_di'][-1]):
        print (f'Ticker: {ticker}; Monthly directional trend is negative')
    elif (df_mo['adx'][-1] < df_mo['ADX_trend'][-1]):
        print (f'Ticker: {ticker}; Monthly directional trend is neutral or sideways')
    time.sleep(1)

Ticker: INMD; Monthly stacked EMA
 is in the positive direction
Ticker: INMD; Monthly directional trend is positive
Ticker: ASAN; Monthly stacked EMA
 is in the positive direction
Ticker: ASAN; Monthly directional trend is positive
Ticker: ATKR; Monthly stacked EMA
 is in the positive direction
Ticker: ATKR; Monthly directional trend is positive
Ticker: SWAV; Monthly stacked EMA
 is in the positive direction
Ticker: SWAV; Monthly directional trend is positive
Ticker: LSPD; Monthly stacked EMA
 is in the positive direction
Ticker: LSPD; Monthly directional trend is positive
Ticker: CELH; Monthly stacked EMA
 is in the positive direction
Ticker: CELH; Monthly directional trend is positive
Ticker: MRNA; Monthly stacked EMA
 is in the positive direction
Ticker: MRNA; Monthly directional trend is positive
Ticker: BILL; Monthly stacked EMA
 is in the positive direction
Ticker: BILL; Monthly directional trend is positive
Ticker: UPST; Monthly stacked EMA
 is in the positive direction
Ticker: 

In [11]:
for ticker in tickers:
    # Download historical data as CSV for each stock (makes the process faster)
    stock = yf.Ticker(ticker)
    df_hr = stock.history(period="3mo", interval='1h')
    df_hr.to_csv(f'{ticker}_hr.csv')    
    m_avg_hr = df_hr['Close'].ewm(span=length).mean()
    df_hr['Moving average'] = m_avg_hr
    # standard deviation
    m_std_hr = df_hr['Close'].rolling(window=length).std()
    # upper Bollinger Bands
    df_hr['upper_BB'] = m_avg_hr + mult * m_std_hr
    # lower Bollinger Bands 
    df_hr['lower_BB'] = m_avg_hr - mult * m_std_hr
    # calculate Keltner Channel which is a measure of the average true range.  In this case the channels are set to twice the average true range
    # first we need to calculate True Range
    df_hr['tr0'] = abs(df_hr["High"] - df_hr["Low"])
    df_hr['tr1'] = abs(df_hr["High"] - df_hr["Close"].shift())
    df_hr['tr2'] = abs(df_hr["Low"] - df_hr["Close"].shift())
    df_hr['tr'] = df_hr[['tr0', 'tr1', 'tr2']].max(axis=1)
    # moving average of the TR or average true range
    range_ma_hr = df_hr['tr'].ewm(span=length_KC).mean()
    # upper Keltner Channel
    df_hr['upper_KC'] = m_avg_hr + range_ma_hr * mult_KC
    # lower Keltner Channel
    df_hr['lower_KC'] = m_avg_hr - range_ma_hr * mult_KC
    
    df_hr['squeeze_on'] = (df_hr['lower_BB'] > df_hr['lower_KC']) & (df_hr['upper_BB'] < df_hr['upper_KC'])
    df_hr['squeeze_off'] = (df_hr['lower_BB'] < df_hr['lower_KC']) & (df_hr['upper_BB'] > df_hr['upper_KC'])
    
    highest_hr = df_hr['High'].rolling(window = length_KC).max()
    lowest_hr = df_hr['Low'].rolling(window = length_KC).min()
    m1_hr = (highest_hr + lowest_hr) / 2
    df_hr['value'] = (df_hr['Close'] - (m1_hr + m_avg_hr)/2)
    fit_y_hr = np.array(range(0,length_KC))
    df_hr['value'] = df_hr['value'].rolling(window = length_KC).apply(lambda x : np.polyfit(fit_y_hr, x, 1)[0] * (length_KC-1) +
    np.polyfit(fit_y_hr, x, 1)[1], raw=True)

    df_hr[['squeeze_on', 'squeeze_off']] = (df_hr[['squeeze_on', 'squeeze_off']] == True).astype(float)

    df_hr["pct_change"] = df_hr['Close'].pct_change()*100

    conditions_hr =  [
        (df_hr['pct_change'] > 0),
        (df_hr['pct_change'] < 0)
    ]

    label = [1, 0]
    df_hr['profitable?'] = np.select(conditions_hr, label) 
    
    # entry point for long position:
    # 1. black cross becomes gray (the squeeze is released)
    long_cond1_hr = (df_hr['squeeze_off'][-2] == 0) | (df_hr['squeeze_off'][-1] == 1) 
    # 2. bar value is positive => the bar is light green
    long_cond2_hr = df_hr['value'][-1] > 0

    enter_long_hr = long_cond1_hr and long_cond2_hr
    # entry point for short position:
    # 1. black cross becomes gray (the squeeze is released)
    short_cond1_hr = (df_hr['squeeze_off'][-2] == 0) | (df_hr['squeeze_off'][-1] == 1) 
    # 2. bar value is negative => the bar is light red 
    short_cond2_hr = df_hr['value'][-1] < 0
    enter_short_hr = short_cond1_hr and short_cond2_hr

    colors = []
    for ind, val in enumerate(df_hr['value']):
        if val >= 0:
            color = 'blue'
            if val > df_hr['value'][ind-1]:
                color = 'cyan'
                    
        else:
            color = 'yellow'
            if val < df_hr['value'][ind-1]:
                color='red'        
        colors.append(color)    
    df_hr['colors'] = colors
    color_value_hr = df_hr['colors'][-1]
    squeeze_value_hr = df_hr['squeeze_on'][-1]
    
    print (f'Ticker: {ticker}; hourly squeeze_scan\n {squeeze_value_hr},{color_value_hr}')
    time.sleep(1)

Ticker: INMD; hourly squeeze_scan
 1.0,yellow
Ticker: ASAN; hourly squeeze_scan
 0.0,blue
Ticker: ATKR; hourly squeeze_scan
 0.0,red
Ticker: SWAV; hourly squeeze_scan
 0.0,blue
Ticker: LSPD; hourly squeeze_scan
 0.0,red
Ticker: CELH; hourly squeeze_scan
 1.0,yellow
Ticker: MRNA; hourly squeeze_scan
 0.0,red
Ticker: BILL; hourly squeeze_scan
 0.0,blue
Ticker: UPST; hourly squeeze_scan
 0.0,red
Ticker: TASK; hourly squeeze_scan
 0.0,red
Ticker: NET; hourly squeeze_scan
 0.0,blue
Ticker: SE; hourly squeeze_scan
 0.0,yellow
Ticker: DOCS; hourly squeeze_scan
 0.0,red
Ticker: ASML; hourly squeeze_scan
 0.0,cyan
Ticker: WAL; hourly squeeze_scan
 0.0,cyan
Ticker: GNRC; hourly squeeze_scan
 0.0,cyan
Ticker: MDB; hourly squeeze_scan
 1.0,red
Ticker: GLBE; hourly squeeze_scan
 0.0,red
Ticker: DLO; hourly squeeze_scan
 0.0,yellow
Ticker: RGEN; hourly squeeze_scan
 1.0,cyan
Ticker: PRFT; hourly squeeze_scan
 0.0,yellow
Ticker: ZS; hourly squeeze_scan
 0.0,cyan
Ticker: AVTR; hourly squeeze_scan
 1.0

In [ ]:
for ticker in tickers:
    # Download historical data as CSV for each stock (makes the process faster)
    stock = yf.Ticker(ticker)
    df_mo = stock.history(period="10y", interval='1mo')
    df_mo.to_csv(f'{ticker}_mo.csv')
    #Stacked Moving averages, exponentially weighted.  Focusing on Fibonacci numbers here with 8, 21, 34, 55 and 89 day moving averages
    m_avg_08_mo = df_mo['Close'].ewm(span=8).mean()
    m_avg_21_mo = df_mo['Close'].ewm(span=21).mean()
    m_avg_34_mo = df_mo['Close'].ewm(span=34).mean()
    m_avg_55_mo = df_mo['Close'].ewm(span=55).mean()
    m_avg_89_mo = df_mo['Close'].ewm(span=89).mean()

    df_mo['m_avg_08'] = m_avg_08_mo
    df_mo['m_avg_21'] = m_avg_21_mo
    df_mo['m_avg_34'] = m_avg_34_mo
    df_mo['m_avg_55'] = m_avg_55_mo
    df_mo['m_avg_89'] = m_avg_89_mo
    
    df_mo['tr0'] = abs(df_mo["High"] - df_mo["Low"])
    df_mo['tr1'] = abs(df_mo["High"] - df_mo["Close"].shift())
    df_mo['tr2'] = abs(df_mo["Low"] - df_mo["Close"].shift())
    df_mo['tr'] = df_mo[['tr0', 'tr1', 'tr2']].max(axis=1)
    # moving average of the TR or average true range
    range_ma = df_mo['tr'].ewm(span=length_KC).mean()
    # upper Keltner Channel
    df_mo['upper_KC'] = m_avg_mo + range_ma * mult_KC
    # lower Keltner Channel
    df_mo['lower_KC'] = m_avg_mo - range_ma * mult_KC
    
    plus_dm = df_mo["High"].diff()
    minus_dm = df_mo["Low"].diff()
    plus_dm[plus_dm < 0] = 0
    minus_dm[minus_dm > 0] = 0

    #plus_di and minus di show the direction of the trend, if plus_di > minus_di the trend is positive
    #if minus_di > plus_di the trend is negative
    df_mo['atr'] = df_mo['tr'].ewm(span=14).mean()
    df_mo['plus_di'] = 100 * (plus_dm.ewm(alpha = 1/14).mean() / df_mo['atr'])
    df_mo['minus_di'] = abs(100 * (minus_dm.ewm(alpha = 1/14).mean() / df_mo['atr']))
    dx_mo = (abs(df_mo['plus_di'] - df_mo['minus_di']) / abs(df_mo['plus_di'] + df_mo['minus_di'])) * 100
    adx_mo = ((dx_mo.shift(1) * (14 - 1)) + dx_mo) / 14
    df_mo['adx'] = adx_mo.ewm(alpha = 1/14).mean() #ADX>20 is trending ADX<20 not trending
    
    df_mo['ADX_trend'] = 20
    
    
    if (df_mo['m_avg_08'][-1] > df_mo['m_avg_21'][-1] > df_mo['m_avg_34'][-1] > df_mo['m_avg_55'][-1] > df_mo['m_avg_89'][-1]):
        print (f'Ticker: {ticker}; Monthly stacked EMA\n is in the positive direction')
    elif (df_mo['m_avg_08'][-1] < df_mo['m_avg_21'][-1] < df_mo['m_avg_34'][-1] < df_mo['m_avg_55'][-1] < df_mo['m_avg_89'][-1]):
        print (f'Ticker: {ticker}; Monthly stacked EMA\n is in the negative direction')
    else:
        print (f'Ticker: {ticker}; Weekly stacked EMA\n is neutral')
        
    if (df_mo['adx'][-1] > df_mo['ADX_trend'][-1]) and (df_mo['plus_di'][-1] > df_mo['minus_di'][-1]):
        print (f'Ticker: {ticker}; Monthly directional trend is positive')
    elif (df_mo['adx'][-1] > df_mo['ADX_trend'][-1]) and (df_mo['plus_di'][-1] < df_mo['minus_di'][-1]):
        print (f'Ticker: {ticker}; Monthly directional trend is negative')
    elif (df_mo['adx'][-1] < df_mo['ADX_trend'][-1]):
        print (f'Ticker: {ticker}; Monthly directional trend is neutral or sideways')
    time.sleep(1)